# Toronto

## Data preparation

### Import libraries

In [1]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests

### open the source page from wikipedia

In [ ]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
source

### retrieve the table
it's the first table of the page

In [ ]:
soup = BeautifulSoup( source, 'lxml')
table = soup.find("table")
table

In [ ]:
table_body = table.tbody

data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
    
data

### Now we refine that table

delete rows with NA burough  
find neighbors for each burough

In [ ]:
del data[0]

burough_neigh = {}
for d in data:
    #borough not assigned 
    if d[1] == 'Not assigned':
        continue
    if d[1] not in burough_neigh:
        burough_neigh[d[1]] = []
    else:
        burough_neigh[d[1]].append(d[2])

burough_neigh

In [ ]:
reduced_data = { }

for d in data:
    post = d[0]
    bur = d[1]
    neigh = d[2]
    
    if d[1] == 'Not assigned':
        continue
    
    if d[2] is 'Not assigned':
        neigh = burough_neigh[d[1]]
    
    if post not in reduced_data:
        reduced_data[post] = { 'bor': [], 'nei': [] }
    
    if bur not in reduced_data[post]['bor']:
        reduced_data[post]['bor'].append(bur)
    if neigh not in reduced_data[post]['nei']:
        reduced_data[post]['nei'].append(neigh)

reduced_data

In [ ]:
refined_data = { "Postal Code": [], 'Borough': [], 'Neighborhood' :[]}

for k,v in reduced_data.items():
    post = k
    bur = v['bor']
    neigh = v['nei']
    
    if neigh is 'Not assigned':
        neigh = burough_neigh[bur]
    
    refined_data["Postal Code"].append(post)
    refined_data["Borough"].append(bur[0])
    refined_data["Neighborhood"].append(neigh)

### Finally we create the dataframe

In [ ]:
df = pd.DataFrame.from_dict(refined_data)
print(df.shape)
df.head()

## Geolocalisation
Since we're given a csv file for that, we'll just use it. Quicker, simplier and more precise. 

In [ ]:
geo = "Geospatial_Coordinates.csv"

geo_df = pd.read_csv(geo)
print(geo_df.shape)
geo_df.head()

In [ ]:
extended_df = pd.merge(df,geo_df, on="Postal Code")
print(extended_df.shape)
extended_df.head()

## Clustering

First we define the limits,
we'll take the extremes and add a little distance to the borders.

In [ ]:
from math import floor, ceil

latitude = geo_df["Latitude"]
longitude = geo_df["Longitude"]
data_points = [ np.array([i,j]) for i,j in zip(latitude, longitude)]


extension = 0.005
l_min = min(latitude) - extension
l_max = max(latitude) + extension
L_min = min(longitude) - extension
L_max = max(longitude) + extension

l_min, l_max, L_min, L_max

Now we define a function to generate initial centroids randomly
using the limits set previously

In [ ]:
def rand_init(l_min, l_max, L_min, L_max, k):
    l_rand = np.random.uniform(l_min, l_max, k)
    L_rand = np.random.uniform(L_min, L_max, k)

    rand_start = [ np.array([i,j]) for i,j in zip(l_rand,L_rand)]
    return rand_start

rand_init(l_min, l_max, L_min, L_max, 3)

Now we can start clustering  
So we'll start by initializing random centroid and creating a distance matrix  

In [ ]:
def distance( pointA, pointB ):
    return np.linalg.norm( (pointA-pointB) )

def cluster_score(cluster, centroid):
    sum_score = 0
    for point in cluster:
        sum_score += distance( point, centroid )
    return sum_score

def get_score( clusters, centroids ):
    score = 0
    for clus,cent in zip(clusters, centroids):
        score += cluster_score(clus, cent)
    return score

Here i take k = 4, but really there's no particular reasoning behind it.  

In [ ]:
k = 4
centroids = rand_init(l_min, l_max, L_min, L_max, k)

def get_distance_matrix(data_points, centroids, k):
    distance_mat = np.zeros( (len(data_points), k) )
    for i in range(len(data_points)): 
        for j in range(len(centroids)):
            dist = distance(data_points[i], centroids[j])
            distance_mat[i][j] = dist
            
    return distance_mat
        
distance_mat = get_distance_matrix(data_points, centroids, k)

Whenever a centroid is the closest to a given data point, we set that data point as part of the cluster categorized by that centroid  

In [ ]:
def clusterize( data_points, distance_mat, k):  
    clusters = [ [] for n in range(k)]
    for i in range(len(data_points)):
        d = distance_mat[i]
        ind = np.argmin(d)
        c_size.append(ind)
        clusters[ind].append(data_points[i])
    return clusters
    
clusters = clusterize(data_points, distance_mat, k)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import random

colors = np.random.rand(k,3)

plt.figure(figsize=(20,10))
#plt.scatter( latitude, longitude )
plt.scatter( [x[0] for x in centroids], [y[1] for y in centroids], c= colors, marker = 's', s=80)

for i in range(len(clusters)):
    plt.scatter( [x[0] for x in clusters[i]], [y[1] for y in clusters[i]], c= [colors[i]]*len(clusters[i]))
    

score = get_score(clusters, centroids)
score

Then we'll move the centroid to the mean of their respective clusters and recalculate a new score

In [ ]:
def recenter_centroids(clusters, centroids):
    for i in range(len(centroids)):
        if(len(clusters[i]) == 0):
            continue
        m_lat = np.array([x[0] for x in clusters[i]]).mean()
        m_lon = np.array([y[1] for y in clusters[i]]).mean()
        centroids[i] = np.array( [m_lat,m_lon] )


In [ ]:
recenter_centroids(clusters, centroids)
        
plt.figure(figsize=(20,10))
plt.scatter( [x[0] for x in centroids], [y[1] for y in centroids], c= colors, marker = 's', s=100)
for i in range(len(clusters)):
    plt.scatter( [x[0] for x in clusters[i]], [y[1] for y in clusters[i]], c= [colors[i]]*len(clusters[i]), marker='.')

And we'll keep doing that until the change in score is either not significant, worse or doesn't change.  
We'll still set an iteration limit at 10000 to prevent it to take too long

In [ ]:
def repeat_until_good(score, centroids, data_points, k):
    for it in range(10000):
        distance_matrix = get_distance_matrix(data_points, centroids,k)
        clusters = clusterize(data_points, distance_matrix, k)
        new_score = get_score(clusters, centroids)
        difference = score - new_score
        if difference < 0: #worse score
            break
        elif difference < eps: # not significant
            break 
        else:
            score = new_score
        recenter_centroids(clusters, centroids)
    
    return centroids, clusters, score

In [ ]:
eps = 1e-5

centroids, clusters, score = repeat_until_good(score, centroids, data_points, k)
    
plt.figure(figsize=(20,10))
plt.scatter( [x[0] for x in centroids], [y[1] for y in centroids], c= colors, marker = 's', s=100)
for i in range(len(clusters)):
    plt.scatter( [x[0] for x in clusters[i]], [y[1] for y in clusters[i]], c= [colors[i]]*len(clusters[i]), marker='.')
    
score

We'll repeat the entire process multiple times to diminish the initialization bias.  
Let's do this 1000 times

In [ ]:
eps = 1e-5
k = 3

saves = []

for iteration in range(1000):
    centroids = rand_init(l_min, l_max, L_min, L_max, k) 
    distance_mat = get_distance_matrix(data_points, centroids, k)
    clusters = clusterize(data_points, distance_mat, k)

    centroids, clusters, score = repeat_until_good(score, centroids, data_points, k)
    saves.append( (score, centroids) )
    

We'll take the best solution, thus the minimal score which means the least distance between centroids and data points.

In [ ]:
best_score, best_centroids = min(saves, key = lambda t: t[0])

best_distance_mat = get_distance_matrix(data_points, best_centroids, k)
best_clusters = clusterize(data_points, best_distance_mat, k)

In [ ]:
colors = np.random.rand(k,3)

plt.figure(figsize=(20,10))
plt.scatter( [x[0] for x in best_centroids], [y[1] for y in best_centroids], c= colors, marker = 's', s=100)
for i in range(len(clusters)):
    plt.scatter( [x[0] for x in best_clusters[i]], [y[1] for y in best_clusters[i]], c= [colors[i]]*len(best_clusters[i]), marker='.')
    
best_score